In [1]:
import pandas as pd
import numpy as np

school_data_to_load = "schools_complete.csv"
student_data_to_load = "students_complete.csv"

school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

pyschool = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
pyschool.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [23]:
pyschool['Student ID1'] = pyschool['Student ID']
pyschool.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget,Student ID1
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,0
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,1
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,2
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635,3
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635,4


In [46]:
pyschool.count().head(12)

Student ID       39170
student_name     39170
gender           39170
grade            39170
school_name      39170
reading_score    39170
math_score       39170
School ID        39170
type             39170
size             39170
budget           39170
Student ID1      39170
dtype: int64

In [24]:
print(pyschool.columns.values)

['Student ID' 'student_name' 'gender' 'grade' 'school_name'
 'reading_score' 'math_score' 'School ID' 'type' 'size' 'budget'
 'Student ID1']


In [25]:
total_school = pyschool['School ID'].nunique()
total_student = pyschool['Student ID'].nunique()
total_budget = (pyschool.groupby(['School ID']).mean()).reset_index()
total_budget1 = total_budget['budget'].sum()
avg_math = pyschool['math_score'].mean()
avg_read = pyschool['reading_score'].mean()
p_pass_math = pyschool.query('math_score >=70')["Student ID"].nunique()/ total_student*100
p_pass_read = pyschool.query('reading_score >=70')['Student ID'].nunique()/ total_student*100
pass_rate = (p_pass_math + p_pass_read)/2

district_summary = pd.DataFrame({"Total Schools":[total_school],
                       "Total Students":[total_student],
                       "Total Budget":[total_budget1],
                       "Average Math Score":[avg_math],
                        "Average Reading Score":[avg_read],
                        "% Passing Math":[p_pass_math],
                        "% Passing Reading":[p_pass_read],
                        "% Overall Passing Rate":[pass_rate]})
district_summary1 = district_summary.style.format({'Total Budget': "${:,.2f}"})
district_summary1

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,"$24,649,428.00",78.9854,81.8778,74.9809,85.8055,80.3932


In [40]:
school_summary = (pyschool.groupby(['school_name', 'type']).agg({'Student ID':'nunique', 'budget':'mean',
                                                           'math_score':'mean', 'reading_score':'mean'})).reset_index()
school_summary['Per Student Budget'] = (school_summary['budget'])/(school_summary['Student ID'])
school_summary


,school_name,type,Student ID,budget,math_score,reading_score,Per Student Budget
0,Bailey High School,District,4976,3124928,77.048432,81.033963,628.0
1,Cabrera High School,Charter,1858,1081356,83.061895,83.975780,582.0
2,Figueroa High School,District,2949,1884411,76.711767,81.158020,639.0
3,Ford High School,District,2739,1763916,77.102592,80.746258,644.0
4,Griffin High School,Charter,1468,917500,83.351499,83.816757,625.0
5,Hernandez High School,District,4635,3022020,77.289752,80.934412,652.0
6,Holden High School,Charter,427,248087,83.803279,83.814988,581.0
7,Huang High School,District,2917,1910635,76.629414,81.182722,655.0
8,Johnson High School,District,4761,3094650,77.072464,80.966394,650.0
9,Pena High School,Charter,962,585858,83.839917,84.044699,609.0


In [98]:
# school_summary1 = pyschool.groupby(['school_name', 'type']).math_score.agg({'%Passing Math': lambda x: (x>=70).count()})
school_summary1 = pyschool.groupby(['school_name', 'type'])
s_stuid_count = school_summary1['Student ID'].nunique()               
s_pass_math = school_summary1.math_score.apply(lambda x: (x>=70).value_counts(normalize=True))*100
s_pass_math1 = s_pass_math.to_frame().reset_index()
s_pass_read = school_summary1.reading_score.apply(lambda x: (x>=70).value_counts(normalize=True))*100
s_pass_read1 = s_pass_read.to_frame().reset_index()
s_pass_read1

,school_name,type,level_2,reading_score
0,Bailey High School,District,True,81.933280
1,Bailey High School,District,False,18.066720
2,Cabrera High School,Charter,True,97.039828
3,Cabrera High School,Charter,False,2.960172
4,Figueroa High School,District,True,80.739234
5,Figueroa High School,District,False,19.260766
6,Ford High School,District,True,79.299014
7,Ford High School,District,False,20.700986
8,Griffin High School,Charter,True,97.138965
9,Griffin High School,Charter,False,2.861035


In [ ]:
# The below each broken by gender
#   * Purchase Count
#   * Average Purchase Price
#   * Total Purchase Value
#   * Average Purchase Total per Person by Gender

purch_count = (gender_group['Purchase ID'].count()).reset_index()
avg_purch_price = (gender_group['Price'].mean()).reset_index()
total_purch_value = (gender_group['Price'].sum()).reset_index()
avg_purch_person = ((gender_group['Price'].sum()) / gender_count).reset_index()
purch_gender = pd.merge((pd.merge(pd.merge(purch_count, avg_purch_price, on='Gender'), 
                                 total_purch_value,on='Gender')), avg_purch_person, on='Gender')
purch_gender.set_index('Gender', inplace=True)
purch_gender.columns = ['Purchase Count', 'Average Purchase Price', 'Total Purchase Value', 'Avg Total Purchase per Person']
purch_gendera = purch_gender.style.format({'Average Purchase Price': "${:.2f}",
                                             'Total Purchase Value': "${:.2f}",
                                             'Avg Total Purchase per Person': "${:.2f}"})
purch_gendera

In [ ]:
print(pymoli['Age'].max())
print(pymoli['Age'].min())

In [ ]:
bins = [0, 9, 14, 19, 24, 29, 34, 39, 50]
age_demograph = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
pd.cut(pymoli['Age'], bins, labels=age_demograph).head()

In [ ]:
pymoli["Age View"] = pd.cut(pymoli["Age"], bins, labels=age_demograph)
pymoli.head()

In [ ]:
age_output = pymoli.groupby('Age View')
age_output1 = age_output['SN'].nunique()
age_output2 = ((age_output1/total_player)*100).reset_index()
age_output3 = age_output['SN'].nunique().reset_index()
age_output4 = pd.merge(age_output3, age_output2, on='Age View')
age_output4.set_index('Age View', inplace=True)
age_output4.columns = ['Total Count', 'Percentage of Players']
age_summary = age_output4.style.format({'Percentage of Players': "%{:.2f}"})
age_summary

In [ ]:
# The below each broken into bins of 4 years (i.e. &lt;10, 10-14, 15-19, etc.)40
#   * Purchase Count
#   * Average Purchase Price
#   * Total Purchase Value
#   * Average Purchase Total per Person by Age Group

age_tally = (pymoli.groupby(['Age View']).agg({'Purchase ID':'count', 'Price':'mean', 'Price1':'sum'})).reset_index()
age_atppp = ((age_output['Price'].sum())/age_output1).reset_index()
age_tally1 = pd.merge(age_tally, age_atppp, on ='Age View')
age_tally1.set_index('Age View', inplace=True)
age_tally1.columns = ['Purchase Count', 'Average Purchase Price', 'Total Purchase Value', 'Avg Total Purchase per Person']
age_tally2 = age_tally1.style.format({'Average Purchase Price': "${:.2f}",
                                       'Total Purchase Value': "${:.2f}",
                                       'Avg Total Purchase per Person': "${:.2f}"})
age_tally2

In [ ]:
# Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
#   * SN
#   * Purchase Count
#   * Average Purchase Price
#   * Total Purchase Value

spender_gr = pymoli.groupby(['SN']).agg({'Purchase ID':'nunique', 'Price':'mean', 'Price1':'sum'})
spender_gr.columns = ['Purchase Count','Average Purchase Price','Total Purchase Value']
spender_gr1 = (spender_gr.sort_values(['Total Purchase Value'],ascending=False))
spender_gr1['Average Purchase Price'] = spender_gr1['Average Purchase Price'].astype(float).map("${:,.2f}".format)
spender_gr1['Total Purchase Value'] = spender_gr1['Total Purchase Value'].astype(float).map("${:,.2f}".format)
spender_gr1.head(5)

In [ ]:
# Identify the 5 most popular items by purchase count, then list (in a table):
#   * Item ID
#   * Item Name
#   * Purchase Count
#   * Item Price
#   * Total Purchase Value

itemid_gr = pymoli.groupby(['Item ID', 'Item Name']).agg({'Purchase ID':'count', 'Price':'mean', 'Price1': 'sum'})
itemid_gr.columns = ['Purchase Count','Item Price','Total Purchase Value']
itemid_gr2 = (itemid_gr.sort_values(['Purchase Count'],ascending=False))
itemid_gr2['Item Price'] = itemid_gr2['Item Price'].astype(float).map("${:,.2f}".format)
itemid_gr2['Total Purchase Value'] = itemid_gr2['Total Purchase Value'].astype(float).map("${:,.2f}".format)
itemid_gr2.head(5)

In [ ]:
# Identify the 5 most profitable items by total purchase value, then list (in a table):
#   * Item ID
#   * Item Name
#   * Purchase Count
#   * Item Price
#   * Total Purchase Value

profit_gr = pymoli.groupby(['Item ID', 'Item Name']).agg({'Purchase ID':'count', 'Price':'mean', 'Price1': 'sum'})
profit_gr.columns = ['Purchase Count','Item Price','Total Purchase Value']
profit_gr2 = (profit_gr.sort_values(['Total Purchase Value'],ascending=False))
profit_gr2['Item Price'] = profit_gr2['Item Price'].astype(float).map("${:,.2f}".format)
profit_gr2['Total Purchase Value'] = profit_gr2['Total Purchase Value'].astype(float).map("${:,.2f}".format)
profit_gr2.head(5)